In [0]:
%python
## Import libraries
import pyspark as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *
import pandas as pd
import json


In [0]:
# Retrieve the parameter values
job_id = dbutils.widgets.get("job_id")
usecase_id = dbutils.widgets.get("usecase_id")

# Print the parameter values
print(f"job_id: {job_id}")
print(f"usecase_id: {usecase_id}")

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-3683226913856893> in <cell line: 2>()
      1 # Retrieve the parameter values
----> 2 job_id = dbutils.widgets.get("job_id")
      3 usecase_id = dbutils.widgets.get("usecase_id")
      4 
      5 # Print the parameter values

/databricks/python_shell/dbruntime/WidgetHandlerImpl.py in get(self, name)
     40         :return: Current value of the widget or default value
     41         """
---> 42         return self._notebookArguments.getArgument(name, self._entry_point.getCurrentBindings())
     43 
     44     def getArgument(self, name, defaultValue=None):

/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1319 
   1320         answer = self.gateway_client.send_command(command)
-> 1321         return_value = get_return_value(
   1322             answer, self.gateway_client, s

In [0]:
class Mandatory_Check:
  def __init__(self):
    self.query = "Query About to fill"
    self.sql_df = pd.DataFrame()
    self.data_json = {"Data":[]}  
    self.usecase_id = None  
  
  def logger(self, message):
    print(message)
    
  def set_query_string(self, usecase_parm):
    query =  f"""UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, REASON_VOD__C is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, REASON_VOD__C is not present')
  END
)
WHERE ( '{usecase_parm}' in  (512,514) and usecase_id ='{usecase_parm}' AND (REASON_VOD__C IS NULL OR trim(REASON_VOD__C) = '')
AND (case when usecase_id  In ('512') then CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}'))
when usecase_id IN ('514') then CAST(created_date as DATE) >= current_date end)
AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, None of the mandatory columns account_vod__c/HCP_OMNI_ID/HCP_NPI is present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, None of the mandatory columns account_vod__c/HCP_OMNI_ID/HCP_NPI is present')
  END
)
WHERE ('{usecase_parm}' = 511  and usecase_id ='{usecase_parm}' AND (account_vod__c IS NULL OR trim(account_vod__c) = '') AND (HCP_OMNI_ID IS NULL OR trim(HCP_OMNI_ID) = '') AND (trim(HCP_NPI) = '' OR HCP_NPI IS NULL) AND CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, account_vod__c is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, ACCOUNT_VOD__C is not present')
  END
)
WHERE 
'{usecase_parm}'in (511,512,513,514)
and usecase_id ='{usecase_parm}' 
AND (account_vod__c IS NULL OR trim(account_vod__c) = '') 
AND (case when '{usecase_parm}'in (513,514) then CAST(created_date as DATE) = current_date  
          when '{usecase_parm}'in (511,512) then (CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0)))) end);

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record,HCP_OMNI_ID  is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, HCP_OMNI_ID is not present')
  END
)
WHERE 
'{usecase_parm}' in (512,513,514)
and usecase_id ='{usecase_parm}' 
AND (HCP_OMNI_ID IS NULL OR trim(HCP_OMNI_ID) = '') 
AND (case when '{usecase_parm}'in (513) then CAST(created_date as DATE) = current_date  
        when '{usecase_parm}'in (512,514) then (CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0)))) end);

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record,HCP_NPI  is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, HCP_NPI is not present')
  END
)
WHERE 
'{usecase_parm}' in (512,513,514)
and usecase_id ='{usecase_parm}' 
AND (HCP_NPI IS NULL OR trim(HCP_NPI) = '') 
AND (case when '{usecase_parm}'in (513,514) then CAST(created_date as DATE) = current_date  
        when '{usecase_parm}'in (512) then (CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0)))) end);

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, USECASE_ID is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, USECASE_ID is not present')
  END
)
WHERE  (USECASE_ID IS NULL OR trim(USECASE_ID) = '')  ;

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, HCP_ZIP_CODE is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, HCP_ZIP_CODE is not present')
  END
)
WHERE
'{usecase_parm}' in (511,512,513)
and usecase_id ='{usecase_parm}' 
AND ( HCP_ZIP_CODE is null or trim(HCP_ZIP_CODE) = '') 
AND (case when '{usecase_parm}'in (513) then CAST(created_date as DATE) = current_date  
        when '{usecase_parm}'in (511,512) then (CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0)))) end);

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, CALL_OBJ_ON_BY_DEFAULT_VOD__C is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, CALL_OBJ_ON_BY_DEFAULT_VOD__C  is not present')
  END
)
WHERE ('{usecase_parm}' in (511,512) and usecase_id ='{usecase_parm}' AND (CALL_OBJ_ON_BY_DEFAULT_VOD__C is null or trim(CALL_OBJ_ON_BY_DEFAULT_VOD__C) = '') AND CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, CALL_OBJ_RECORD_TYPE_VOD__C is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, CALL_OBJ_RECORD_TYPE_VOD__C is not present')
  END
)
WHERE ( '{usecase_parm}' in (511,512) and usecase_id ='{usecase_parm}' AND ( CALL_OBJ_RECORD_TYPE_VOD__C is null or trim(CALL_OBJ_RECORD_TYPE_VOD__C) = '')  AND CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, DISPLAY_DISMISS_VOD__C is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, DISPLAY_DISMISS_VOD__C is not present')
  END
)
WHERE ('{usecase_parm}' in (511,512) and usecase_id ='{usecase_parm}' AND ( DISPLAY_DISMISS_VOD__C is null or trim(DISPLAY_DISMISS_VOD__C) = '')  AND CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, DISPLAY_MARK_AS_COMPLETE_VOD__C is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, DISPLAY_MARK_AS_COMPLETE_VOD__C is not present')
  END
)
WHERE ('{usecase_parm}' in (511,512) and usecase_id ='{usecase_parm}' AND ( DISPLAY_MARK_AS_COMPLETE_VOD__C is null or trim(DISPLAY_MARK_AS_COMPLETE_VOD__C) = '')  AND CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, DISPLAY_SCORE_VOD__C is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, DISPLAY_SCORE_VOD__C is not present')
  END
)
WHERE ('{usecase_parm}' in (511,512) and usecase_id ='{usecase_parm}' AND (DISPLAY_SCORE_VOD__C is null or trim(DISPLAY_SCORE_VOD__C) = '')  AND CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, EXPIRATION_DATE_VOD__C is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, EXPIRATION_DATE_VOD__C is not present')
  END
)
WHERE ('{usecase_parm}' in (511,512) and usecase_id ='{usecase_parm}' AND ( EXPIRATION_DATE_VOD__C is null or trim(EXPIRATION_DATE_VOD__C) = '')  AND CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, NO_HOMEPAGE_VOD__C is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, NO_HOMEPAGE_VOD__C is not present')
  END
)
WHERE ('{usecase_parm}' in (511,512) and usecase_id ='{usecase_parm}' AND ( NO_HOMEPAGE_VOD__C is null or trim(NO_HOMEPAGE_VOD__C) = '') AND  CAST(created_date as DATE)>= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, POSTED_DATE_VOD__C is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, POSTED_DATE_VOD__C is not present')
  END
)
WHERE ('{usecase_parm}' in (511,512) and usecase_id ='{usecase_parm}' AND ( POSTED_DATE_VOD__C is null or trim(POSTED_DATE_VOD__C) = '') AND  CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, PRIORITY_VOD__C is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, PRIORITY_VOD__C is not present')
  END
)
WHERE ('{usecase_parm}' in (511,512) and usecase_id ='{usecase_parm}' AND ( PRIORITY_VOD__C is null or trim(PRIORITY_VOD__C) = '') AND CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, RECORDTYPEID_SUGGESTION is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, RECORDTYPEID_SUGGESTION is not present')
  END
)
WHERE ('{usecase_parm}' in (511,512) and usecase_id ='{usecase_parm}' AND (RECORDTYPEID_SUGGESTION is null or trim(RECORDTYPEID_SUGGESTION) = '') AND CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, RECORD_TYPE_NAME_VOD__C_SUGGEST is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, RECORD_TYPE_NAME_VOD__C_SUGGEST is not present')
  END
)
WHERE ('{usecase_parm}' in (511,512) and usecase_id ='{usecase_parm}' AND ( RECORD_TYPE_NAME_VOD__C_SUGGEST is null or  trim(RECORD_TYPE_NAME_VOD__C_SUGGEST) = '') AND CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, SUPPRESS_REASON_VOD__C is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, SUPPRESS_REASON_VOD__C is not present')
  END
)
WHERE ('{usecase_parm}' in (511,512) and usecase_id ='{usecase_parm}' AND (SUPPRESS_REASON_VOD__C is null or trim(SUPPRESS_REASON_VOD__C) = '')AND  CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, TITLE_VOD__C is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, TITLE_VOD__C is not present')
  END
)
WHERE ('{usecase_parm}' in (511,512,514) and usecase_id ='{usecase_parm}' AND ( TITLE_VOD__C is null or trim(TITLE_VOD__C) = '') AND 
(case when usecase_id In ('514') then CAST(created_date as DATE) >= current_date else (CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}'))) end)  
AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, CREATED_BY is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, CREATED_BY is not present')
  END
)
WHERE ('{usecase_parm}' in (511,512) and usecase_id ='{usecase_parm}' AND ( CREATED_BY is null or trim(CREATED_BY) = '') AND CAST(created_date as DATE) >= date_add(current_date, -(SELECT t.hist_retention FROM com_us_alyt_ngebox.metadata_usecase t WHERE t.usecase_id = '{usecase_parm}')) AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, RecordTypeId is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, RecordTypeId is not present')
  END
)
WHERE ('{usecase_parm}' in (513) and usecase_id ='{usecase_parm}' AND ( RecordTypeId is null or trim(RecordTypeId) = '') AND CAST(created_date as DATE)= current_date);

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, TKD_Activity_Description__c is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, TKD_Activity_Description__c is not present')
  END
)
WHERE ('{usecase_parm}' in (513) and usecase_id ='{usecase_parm}' AND ( TKD_Activity_Description__c is null or trim(TKD_Activity_Description__c) = '') AND CAST(created_date as DATE)= current_date);

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, TKD_Activity_Name__c is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, TKD_Activity_Name__c is not present')
  END
)
WHERE ('{usecase_parm}' in (513) and usecase_id ='{usecase_parm}' AND ( TKD_Activity_Name__c is null or trim(TKD_Activity_Name__c) = '') AND CAST(created_date as DATE)= current_date);

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, External_ID is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, External_ID is not present')
  END
)
WHERE
'{usecase_parm}' in (513,514)
and usecase_id ='{usecase_parm}' 
AND ( External_ID is null or trim(External_ID) = '') 
AND (case when '{usecase_parm}'in (513) then CAST(created_date as DATE) = current_date  
        when '{usecase_parm}'in (514) then (CAST(created_date as DATE) = current_date AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0)))) end);

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, suggest_external_id_vod__c is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, suggest_external_id_vod__c is not present')
  END
)
WHERE ('{usecase_parm}' in (514) and usecase_id ='{usecase_parm}' AND ( suggest_external_id_vod__c is null or trim(suggest_external_id_vod__c) = '') AND CAST(created_date as DATE) = current_date);

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, ownerid is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, ownerid is not present')
  END
)
WHERE ('{usecase_parm}' in (514) and usecase_id ='{usecase_parm}' AND ( ownerid is null or trim(ownerid) = '') AND  (CAST(created_date as DATE)= current_date AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0)))));

UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET Not_Sent_To_Veeva_Reason = (
  CASE WHEN ISNULL(Not_Sent_To_Veeva_Reason) = true THEN 'Invalid Record, suggestion_survey_vod__c is not present'
  ELSE CONCAT(Not_Sent_To_Veeva_Reason, '|', 'Invalid Record, suggestion_survey_vod__c is not present')
  END
)
WHERE ('{usecase_parm}' in (514) and usecase_id ='{usecase_parm}' AND ( suggestion_survey_vod__c is null or trim(suggestion_survey_vod__c) = '') AND  (CAST(created_date as DATE)= current_date AND ngebox_record_id IN (select p.ngebox_record_id from com_us_alyt_ngebox.ngebox_suggestion_requests_processed p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id ='{usecase_parm}' and ((s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0)))));"""
    self.query = query    

  def run_query_and_set_sqldf(self, usecase_id):
      self.usecase_id = usecase_id
      query_list = self.query.split(";")
      self.sql_df = [] 
      for curr_query in query_list:
          if curr_query.strip() == "":
              continue
          curr_query_with_param = curr_query.replace("Usecase_parm", usecase_id)
          sql_df = sqlContext.sql(curr_query_with_param).collect()
          self.sql_df.extend(sql_df)
    
  def format_sqldf_to_json(self):
    sql_output_rowlist = []
    for curr_row in self.sql_df: 
      sql_output_rowlist.append (curr_row.asDict(True))
    sql_df_new = pd.DataFrame(sql_output_rowlist)
    sql_jsonstr = sql_df_new.to_json(orient="records") # type(result) -> str
    sql_jsonparse = json.loads(sql_jsonstr) # type(parsed_json) -> list
    nb_json = {"data":sql_jsonparse} # type(metadata_json) -> dict
    self.nb_json = nb_json
    
  def construct_microservice_response(self):
    resp = {
      "nge_response": {
        "status": 200,
        "body": self.nb_json
      }
    }
    self.response = resp

In [0]:
usecase_id = dbutils.widgets.get("usecase_id")
nb_obj = Mandatory_Check()
nb_obj.set_query_string(usecase_parm=usecase_id)
nb_obj.run_query_and_set_sqldf(usecase_id=usecase_id)
nb_obj.format_sqldf_to_json()
nb_obj.construct_microservice_response()
dbutils.notebook.exit(nb_obj.response)